In [14]:
# import the necessary packages
import numpy as np
import cv2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
import math
import os
import time
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

class ExtractSift:
    def __init__(self, bins):
        # store the number of bins the histogram will use
        self.bins = bins
 
    def describe(self, image):
        hist = cv2.calcHist([image], [0, 1, 2],
        None, self.bins, [0, 256, 0, 256, 0, 256])
        hist = cv2.normalize(hist)
        return hist.flatten()
        
        #previous code endsflatten
        #descriptor_mat=[]
        #descriptor_mat = np.zeros()
        #sift=cv2.SIFT(4)        
        #image=cv2.resize(image, (500, 300), interpolation=cv2.INTER_AREA)
        #gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #gray=cv2.equalizeHist(gray)
        #kp, des=sift.detectAndCompute(gray,None)        
        #descriptor_mat.append(des)       
        #descriptor_mat= np.asarray(descriptor_mat)
        #descriptor_mat=descriptor_mat.flatten()        
        #return descriptor_mat
        #svd = TruncatedSVD(n_components=0, algorithm='randomized', n_iter=3, random_state=None, tol=0.0)
        #pca = PCA(n_components=4)
        #for pic in Training:
        #kp, des = cv2.SIFT().detectAndCompute(image, None)
        #descriptors = np.append(descriptors, des)
        #svd.fit_transform(des)
        #newmatrix = svd.transform(des)
        #newmatrix.shape
        #print "New Matrix value"
        #print newmatrix.shape
        #des = pca.transform(des)
        #print "inside des"
        #print des
        #print "New Matrix Again"
        #newmatrix = newmatrix.flatten()
        #print newmatrix.shape
        #return des.flatten()
    
    


#pca.fit(X_scaled)
#X_pca = pca.transform(X_scaled)
    
import argparse
import cPickle
import glob
import cv2
from sklearn.preprocessing import scale
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import LSHForest as LSHForest
import matplotlib.pyplot as plt

#initialize the index dictionary to store our our quantifed
# images, with the 'key' of the dictionary being the image
# filename and the 'value' our computed features
index = {}

def createVocabulary(data,k,itr):
    clust_centers, labels = mykmeansplusplus(data,k,itr)
    f = open("C:/Index-Project/ankur.txt", "w")
    f.write(cPickle.dumps(index))
    f.close()
    return clust_centers

def mykmeansplusplus (points,k,itr):
    data_x_shape = points.shape[0]
    index = np.random.randint(data_x_shape, size =1)
    initial_clust_center = points[index]
    initialCentroid = points[index]
    weight = np.zeros((points.shape[0]))
    index = np.zeros((points.shape[0]))

    for i in range(0,(k - 1)):
        Y= cdist(points, initialCentroid, metric='euclidean', p=2, V=None, VI=None, w=None)
        min_dist = np.amin(Y, axis=1) 
        min_center = np.argmin(Y, axis=1)
        min_dist_sum = np.sum(min_dist, axis=0)
        min_squared_dist_sum = np.sum(min_dist**2, axis=0)
        min_dist = min_dist**2
        min_dist_prob = min_dist / min_squared_dist_sum
        next_index = np.random.choice(points.shape[0],1,p=min_dist_prob)
        initial_clust_center = np.concatenate((initial_clust_center,points[next_index]),axis=0)
        
    initial_class = np.zeros((points.shape[0]))
    for j in range(0,itr):
        Z=cdist(points, initial_clust_center, metric='euclidean', p=2, V=None, VI=None, w=None)
        target_dist = np.amin(Z, axis=1)  
        target_class = np.argmin(Z, axis=1)
        for i in range(0,k-1):
            values = points[target_class == i]
            #print(values)
            #print(i)
            initialCentroid[i,:] = np.mean(values, axis = 0)
        if np.array_equal(initial_class,target_class):
            break
        else:
            initial_class = target_class

    return initial_clust_center, target_class  

# initialize our image descriptor -- a 3D RGB histogram with
# 8 bins per channel
desc = ExtractSift([8, 8, 8])
feature_size = math.pow(8, 3)
def read():
    path = "C:/Users/ankur/Downloads/jpg/"
    print("Path inside read")
    print path
    imlist = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]
    
    
    featuresCollect = np.zeros([len(imlist), feature_size])
    i=0
    # use glob to grab the image paths and loop over them
    for imagePath in glob.glob("C:/Users/ankur/Downloads/jpg" + "/*.jpg"):
        # extract our unique image ID (i.e. the filename)
        k = imagePath[imagePath.rfind("/") + 1:]
        # load the image, describe it using our RGB histogram
        # descriptor, and update the index
        image = cv2.imread(imagePath)
        features = desc.describe(image)
        preprocessing.scale(features,axis = 0, with_mean=True, with_std=False)
        featuresCollect[i] = features 
        #print("Added a feature desc")
        i=i+1
        index[k] = features
    clusters = createVocabulary(featuresCollect,2,100)

    

    

print "Time taken by read"
t0 = time.time()
#read()
print (time.time()-t0)
    
print "Done with Vocab"
#print vocab    

def myvlad(local_descriptors, centroids):
    V = np.zeros([centroids.shape[0],local_descriptors.shape[1]])
    distances = pairwise_distances(local_descriptors, centroids, metric='euclidean')
    clusters = np.argmin(distances,axis=1)
    for iter, center in enumerate(centroids):
        points_belonging_to_cluster = local_descriptors[clusters == iter]
        V[iter] = np.sum(points_belonging_to_cluster - center, axis=0)
        
    return V

print "Now Make VLAD"
#vlad = my_vlad(featuresCollect,vocab)
#print vlad
print "After VLAD"

print "Applying ANN Now"

#approx_neighbors = lshf.kneighbors(vlad,4,return_distance=True)
#print approx_neighbors
    
# we are now done indexing our image -- now we can write our
# index to disk



# import the necessary packages
import numpy as np
 
class VLAD:
    def __init__(self, index):
        # store our index of images
        self.index = index
      
 
    def my_vlad(self, queryFeatures):
        # initialize our dictionary of results
        results = {}
       
        mAP=0.0
        numberOfMatch=0
        c=0
 
        # loop over the index
        for (k, features) in self.index.items():
            c=c+1
            d = self.chi2_distance(features, queryFeatures)
            results[k] = d
            if (d<=1.5):
                mAP=mAP+(1.0/c)
                numberOfMatch=numberOfMatch+1
                
        mAP=mAP/numberOfMatch
           
        print "Mean Avg Prec"
        print mAP
 
        # sort our results, so that the smaller distances (i.e. the
        # more relevant images are at the front of the list)
        results = sorted([(v, k) for (k, v) in results.items()])
        #print ("Array of Distances")
        #print results
        
        
        ####Precision - Recall Curve
        rankedLists = results
        numViewedImages = 5
        for ri in xrange(0, numViewedImages):
            # The first image is the query image itself
            (score, imgId) = results[ri+1]
            #print imageName
            imgPath = "C:/Users/ankur/Downloads/%s" % (imgId)
            #imgId = rankedLists[ri+1]
            
            #imgPath = "C:/Users/ankur/Downloads/%s"+imgId[1]
            plt.subplot(5,numViewedImages/5,ri) 
            #plt.subimage(imread(imgPath))
        plt.show()

        # return our results
        return results
    
 
    def chi2_distance(self, histA, histB, eps = 1e-10):
        # compute the chi-squared distance
        d = 0.5 * np.sum([((a - b) ** 2) / (a + b + eps)
            for (a, b) in zip(histA, histB)])
        # return the chi-squared distance
        return d
    

# import the necessary packages
#from pyimagesearch.searcher import Searcher
import numpy as np
import argparse
import cPickle
import cv2

# load the index and initialize our searcher
index = cPickle.loads(open("C:/Index-Project/ankur.txt").read())
searcher = VLAD(index)


# loop over images in the index -- we will use each one as
# a query image
cv2.startWindowThread()
#for (query, queryFeatures) in index.items():
# perform the search using the current query
print "Time taken by vlad"
t0 = time.time()

query="131500.jpg"
path ="C:/" + "/Anu/%s" % (query)
#print path
image = cv2.imread(path)
#print image
queryFeatures = desc.describe(image)

results = searcher.my_vlad(queryFeatures)

print (time.time()-t0)
 
# load the query image and display itqueryFeatures
#print query
#print queryFeatures

#print path
queryImage = cv2.imread(path)
#print queryImage
#print queryImage
cv2.imshow('Query', queryImage)
cv2.waitKey(0) & 0xFF
#print "query: %s" % (query)
 
# initialize the two montages to display our results --
# we have a total of 25 images in the index, but let's only
# display the top 10 results; 5 images per montage, with
# images that are 400x166 pixels
#montageA = np.zeros((3264*5,2448,3), dtype = "uint8")
#montageB = np.zeros((3264*5,2448,3), dtype = "uint8")

# loop over the top ten results


#Code to output Mean Average Precision



for j in xrange(0, 2):
    # grab the result (we are using row-major order) and
    # load the result image
    (score, imageName) = results[j]
    #print imageName
    path = "C:/Users/ankur/Downloads/%s" % (imageName)
    print "Testing path"
    print path
    result = cv2.imread(path)
    cv2.imshow("Results 1-5", result)
    cv2.waitKey(0) & 0xFF

cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

Time taken by read
0.0
Done with Vocab
Now Make VLAD
After VLAD
Applying ANN Now
Time taken by vlad
Mean Avg Prec
0.00212906287259
16.8660001755
Testing path
C:/Users/ankur/Downloads/jpg\131500.jpg
Testing path
C:/Users/ankur/Downloads/jpg\131501.jpg
